# **Semiparametric SVM training using subgradients in Spark **

#### bla, bla, bla. 

#### We will benchmark the algorithms with data files from UCI:

* **Ripley**, the Ripley dataset
* **Kwok**, the Kwok dataset
* **Twonorm**, the Twonorm dataset
* **Waveform**, the Waveform dataset
* **Covertype**, the Covertype dataset


In [1]:
sc.addPyFile("/export/usuarios01/navia/spark/SVM_spark/common/lib/svm_utils.py")
import svm_utils as SVM
import numpy as np
from pyspark.mllib.regression import LabeledPoint

%matplotlib inline

kfold = 1
kdataset = 1
Niter = 150
Niter = 20

Nnodes = 32
Samplefraction = 0.05

x_tr, y_tr, x_tst, y_tst, sigma, C, NC, name_dataset = SVM.load_data(kdataset)

XtrRDD = sc.parallelize(np.hstack((y_tr, x_tr)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
XtstRDD = sc.parallelize(np.hstack((y_tst, x_tst)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))

auc_tst, elapsed_time = SVM.train_hybridSVM(XtrRDD, XtstRDD, sigma, C, NC, name_dataset, Niter, Samplefraction)

Training the linear SVM model with 20 iterations
0

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 3.0 failed 4 times, most recent failure: Lost task 9.3 in stage 3.0 (TID 160, node84.cluster.tsc.uc3m.es): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/export/workdir/mesos/work/slaves/b7816db4-74d6-4d70-8fe3-b397efddd83b-S31/frameworks/b7816db4-74d6-4d70-8fe3-b397efddd83b-0042/executors/b7816db4-74d6-4d70-8fe3-b397efddd83b-S31/runs/b389ec17-9406-4d33-9f09-81a498378bf8/spark-1.6.0-bin-2.6.0/python/lib/pyspark.zip/pyspark/worker.py", line 98, in main
    command = pickleSer._read_with_length(infile)
  File "/export/workdir/mesos/work/slaves/b7816db4-74d6-4d70-8fe3-b397efddd83b-S31/frameworks/b7816db4-74d6-4d70-8fe3-b397efddd83b-0042/executors/b7816db4-74d6-4d70-8fe3-b397efddd83b-S31/runs/b389ec17-9406-4d33-9f09-81a498378bf8/spark-1.6.0-bin-2.6.0/python/lib/p